In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
df_unfiltered = pd.read_csv('mental-heath-in-tech-2016_20161114.csv')

In [3]:
df_unfiltered.shape

(1433, 63)

In [4]:
missing_percentage = df_unfiltered.isnull().mean() * 100

threshold = 30
columns_to_drop = df_unfiltered.columns[missing_percentage > threshold]
df = df_unfiltered.drop(columns=columns_to_drop)

In [5]:
df.shape

(1433, 48)

In [6]:
df.head()

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up a mental health issue with a potential employer in an interview?,Why or why not?.1,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What country do you work in?,Which of the following best describes your work position?,Do you work

### Mappings

In [11]:
interval_mapping = {  #For company employee number 
    '1-5': 3,
    '6-25': 15.5,
    '26-100': 63,
    '100-500': 300,
    '500-1000': 750,
    'More than 1000': 1500
}
age_mapping = {          #For age of participants
    (18,25): 21.5,
    (26, 39): 32.5,
    (40,59): 49.5,
    (60,70): 65.0
}
replace_dict = {           #for gender of participants
    'male': 1, 'm': 1, 'man': 1,
    'female': 0, 'f': 0, 'women': 0
}

In [12]:
df['How many employees does your company or organization have?'] = df['How many employees does your company or organization have?'].replace(interval_mapping)

In [13]:
median_value = df['How many employees does your company or organization have?'].median()
df['How many employees does your company or organization have?'].fillna(median_value, inplace=True)

In [14]:
mode_age = df['What is your age?'].mode()[0]
for e in range(0,1433):
    if df['What is your age?'][e] < 18 or df['What is your age?'][e] > 70:
        df.loc[e, 'What is your age?'] = mode_age


In [15]:
def map_age(age):
    for age_range, mapped_value in age_mapping.items():
        if age_range[0] <= age <= age_range[1]:
            return mapped_value
    return age  

df['What is your age?'] = df['What is your age?'].apply(map_age)

In [16]:
df['What is your gender?'] = df['What is your gender?'].str.lower().str.strip()

In [17]:
df['What is your gender?'] = df['What is your gender?'].replace(replace_dict)

In [18]:
mode_value = df['What is your gender?'].mode()[0]  # mode() returns a Series, so we take the first value

# Fill NaN values with the mode
df['What is your gender?'].fillna(mode_value, inplace=True)

In [19]:
df['What is your gender?'].isnull().sum()

0

In [20]:
for e in range(len(df)):
    if isinstance(df.loc[e, 'What is your gender?'], str):
        df.loc[e, 'What is your gender?'] = 1

## Binary Features

**Is your employer primarily a tech company/organization?**

**Do you have previous employers?**


In [21]:
from sklearn.impute import SimpleImputer

In [22]:
columns_to_impute = ['Is your employer primarily a tech company/organization?',
                     'Do you have previous employers?']

imput = SimpleImputer(missing_values = np.nan, \
    strategy = 'most_frequent')
imput = imput.fit(df[columns_to_impute])
imput = imput.transform(df[columns_to_impute])
df[columns_to_impute]= imput

## Label encoded columns 



In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
columns_to_LabelEncode = ['If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:', 
                          'Do you know the options for mental health care available under your employer-provided coverage?',
                          'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?',
                          'Have your previous employers provided mental health benefits?', 
                          'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
                          'Do you feel that your employer takes mental health as seriously as physical health?',
                         'Would you feel comfortable discussing a mental health disorder with your coworkers?', 
                         'Do you think that discussing a physical health issue with your employer would have negative consequences?',
                         'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
                         'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
                         'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
                         'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
                         'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
                         'Did your previous employers provide resources to learn more about mental health issues and how to seek help?',
                         'Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?',
                         'Do you think that discussing a mental health disorder with previous employers would have negative consequences?',
                         'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
                         'Would you have been willing to discuss a mental health issue with your previous co-workers?',
                         'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?',
                         'Did you feel that your previous employers took mental health as seriously as physical health?',
                         'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?',
                         'Would you be willing to bring up a physical health issue with a potential employer in an interview?',
                         'Would you bring up a mental health issue with a potential employer in an interview?',
                         'Do you feel that being identified as a person with a mental health issue would hurt your career?',
                         'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?',
                         'How willing would you be to share with friends and family that you have a mental illness?',
                         'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?',
                         'Do you have a family history of mental illness?',
                         'Have you had a mental health disorder in the past?',
                         'Do you currently have a mental health disorder?',
                         'Have you been diagnosed with a mental health condition by a medical professional?',
                         'If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
                         'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?',
                         'Do you work remotely?']


In [25]:
label_encoder = LabelEncoder()


for column in columns_to_LabelEncode:
    #imputing missing values
    imputer = SimpleImputer(strategy='most_frequent')
    
    df[column] = imputer.fit_transform(df[[column]])
    
    df[column] = label_encoder.fit_transform(df[column])

In [26]:
df['Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?'].value_counts()

2    715
0    382
1    336
Name: Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?, dtype: int64

## Label encoding without imputing nan values

**Does your employer provide mental health benefits as part of healthcare coverage?**

**Were you aware of the options for mental health care provided by your previous employers?**

In [27]:
columns_to_LabelEncode =['Does your employer provide mental health benefits as part of healthcare coverage?',
                        'Were you aware of the options for mental health care provided by your previous employers?',
                        ]
label_encoder = LabelEncoder()


for column in columns_to_LabelEncode:
    
    df[column] = label_encoder.fit_transform(df[column])

### Sentimental analysis and feature aggregation

Would you be willing to bring up a physical health issue with a potential employer in an interview?

Why or why not?

Would you bring up a mental health issue with a potential employer in an interview?

Why or why not?.1

In [29]:
from textblob import TextBlob

In [30]:
def get_sentiment(text):
    if pd.isna(text):
        return text  # Leave missing values unchanged
    analysis = TextBlob(str(text))  # Ensure the text is a string
    # Return 1 for positive, 0 for negative
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity < 0:
        return 0

In [31]:
df['Why or why not?'] = df['Why or why not?'].apply(get_sentiment)

In [32]:
df['Why or why not?.1'] = df['Why or why not?.1'].apply(get_sentiment)

In [33]:
columns_to_impute = ['Why or why not?', 'Why or why not?.1']

imput = SimpleImputer(missing_values = np.nan, \
    strategy = 'most_frequent')
imput = imput.fit(df[columns_to_impute])
imput = imput.transform(df[columns_to_impute])
df[columns_to_impute]= imput

In [34]:
df['Why or why not?'].value_counts()

1.0    1207
0.0     226
Name: Why or why not?, dtype: int64

In [35]:
df['Why or why not?.1'].value_counts()

0.0    1145
1.0     288
Name: Why or why not?.1, dtype: int64

In [36]:
import os

In [37]:
current_directory = os.getcwd()

# Define file path
file_path = os.path.join(current_directory, 'clean_data.csv')

# Save DataFrame to CSV
df.to_csv(file_path, index=False)

In [38]:
from sklearn.preprocessing import MinMaxScaler
columns_to_normalize = ['Would you be willing to bring up a physical health issue with a potential employer in an interview?',
                        'Would you bring up a mental health issue with a potential employer in an interview?',
                        'Why or why not?',
                       'Why or why not?.1']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])


In [39]:
weight_ordinal = 0.5
weight_sentiment = 0.5

df['Would you be willing to bring up a physical health issue with a potential employer in an interview?_combined'] = (weight_ordinal * df['Would you be willing to bring up a physical health issue with a potential employer in an interview?']) + (weight_sentiment * df['Why or why not?'])
df['Would you bring up a mental health issue with a potential employer in an interview?_combined'] = (weight_ordinal * df['Would you bring up a mental health issue with a potential employer in an interview?']) + (weight_sentiment * df['Why or why not?.1'])


In [40]:
columns_to_drop= ['Would you be willing to bring up a physical health issue with a potential employer in an interview?',
                  'Would you bring up a mental health issue with a potential employer in an interview?',
                 'Why or why not?.1',
                 'Why or why not?']
df.drop(columns=df[columns_to_drop], inplace=True)

In [165]:
df.head()

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,Do you work remotely?,Would you be willing to bring up a physical health issue with a potential employer in an interview?_combined,Would you bring up a mental health issue with a potential employer in an interview?_combined
0,0,63.0,1.0,2,1,1,1,0,5,1,1,0,2,0,0,1.0,1,1,0,0,0,2,0,1,2,0,0,0,1,4,1,1,2,1,1,0,1,1,32.5,1,2,0.5,0.00
1,0,15.5,1.0,1,2,2